# 🏠 **HabiData: Limpieza y Preprocesamiento de Datos Inmobiliarios**
## Análisis de Precios de Propiedades en Antioquia, Colombia

---

### **📊 Contexto del Proyecto**
Este notebook presenta el proceso completo de **limpieza y preprocesamiento** de un dataset de propiedades inmobiliarias en Colombia, con enfoque específico en **Antioquia**. 

### **🎯 Objetivo**
Preparar un dataset de alta calidad para **predicción de precios** inmobiliarios, aplicando técnicas avanzadas de:
- ✅ Limpieza de datos
- ✅ Validación geográfica  
- ✅ Text mining de descripciones
- ✅ Imputación inteligente de valores faltantes

### **📝 Metodología**
Cada decisión de limpieza está **justificada técnicamente** y documentada para garantizar:
1. **Reproducibilidad** del proceso
2. **Transparencia** en las decisiones 
3. **Calidad científica** del análisis
4. **Optimización** para modelos predictivos

---

### **📋 Estructura del Análisis**
1. **Carga y Configuración Inicial**
2. **Filtrado Geográfico (Antioquia)**  
3. **Evaluación de Calidad de Datos**
4. **Limpieza de Precios Inválidos**
5. **Validación y Corrección Geográfica**
6. **Filtrado por Tipo de Propiedad**
7. **Análisis de Valores Faltantes**
8. **Text Mining de Descripciones**
9. **Estrategia de Integración de Datos**
10. **Análisis de Variables de Superficie**
11. **Tratamiento Final de Valores Faltantes**
12. **Validación y Resumen Final**

---

*Desarrollado como parte del análisis de mercado inmobiliario colombiano*

## 1️⃣ **Carga y Configuración Inicial**

### **🎯 Objetivo de Esta Sección**
Establecer el entorno de trabajo y cargar el dataset inicial de propiedades inmobiliarias colombianas.

### **📚 Justificación de Librerías**
- `pandas`: Manipulación eficiente de datasets grandes
- `numpy`: Operaciones numéricas optimizadas  
- `matplotlib/seaborn`: Visualización de patrones de datos
- `re`: Text mining de descripciones (procesamiento de lenguaje natural)
- `sqlalchemy`: Conexión a base de datos PostgreSQL (opcional)

In [1]:
# ===============================================================
# IMPORTACIÓN DE LIBRERÍAS ESENCIALES
# ===============================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings

# Configuración para optimizar visualización
plt.style.use('default')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Configuración de pandas para mejor visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)

print("✅ Librerías importadas exitosamente")
print("✅ Configuración de visualización establecida")

✅ Librerías importadas exitosamente
✅ Configuración de visualización establecida


In [2]:
# ===============================================================
# CARGA DEL DATASET INICIAL
# ===============================================================

# Ruta al archivo CSV con datos de propiedades colombianas
csv_path = "../data/co_properties.csv"

# Cargar dataset completo
df_original = pd.read_csv(csv_path)

# Información básica del dataset
print("🏠 DATASET CARGADO: Propiedades Inmobiliarias Colombia")
print("=" * 55)
print(f"📊 Dimensiones: {df_original.shape}")
print(f"📋 Registros: {df_original.shape[0]:,}")
print(f"📋 Variables: {df_original.shape[1]}")
print(f"💾 Tamaño en memoria: {df_original.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

# Vista previa de primeras filas
print(f"\n📋 PRIMERAS 3 FILAS:")
df_original.head(3)

🏠 DATASET CARGADO: Propiedades Inmobiliarias Colombia
📊 Dimensiones: (1000000, 25)
📋 Registros: 1,000,000
📋 Variables: 25
💾 Tamaño en memoria: 1482.7 MB

📋 PRIMERAS 3 FILAS:


,id,ad_type,start_date,end_date,created_on,lat,lon,l1,l2,l3,l4,l5,l6,rooms,bedrooms,bathrooms,surface_total,surface_covered,price,currency,price_period,title,description,property_type,operation_type
0,KsjahK62rxcYKXXQjOdkqw==,Propiedad,2020-10-07,2021-10-09,2020-10-07,3.9210,-76.506000,Colombia,Valle del Cauca,NaN,NaN,NaN,NaN,NaN,6.0,7.0,NaN,NaN,1.300000e+09,COP,NaN,Casa Campestre en venta en darien 3469064,"HERMOSA CASA CAMPESTRE, &Aacute;REA 6,000 MT, UBICADA EN LA VIA BUGA - BUENAVENTURA, EN PARCELAC...",Casa,Venta
1,Y+gsBZYq1zu5NoR3V5oUGA==,Propiedad,2020-10-07,2021-01-06,2020-10-07,3.3577,-76.541811,Colombia,Valle del Cauca,Cali,Ciudad Jardín,NaN,NaN,NaN,NaN,7.0,NaN,NaN,2.800000e+09,COP,NaN,Casa en ciudsd jardin,Casa independiente con posiciona en ciudad jardín y hermosos jardines. La casa s eve de amoblada...,Casa,Venta
2,Jpzqxj8/Vgf3Aa5ASxUBNg==,Propiedad,2020-10-07,2020-10-07,2020-10-07,3.3577,-76.541811,Colombia,Valle del Cauca,Cali,Ciudad Jardín,NaN,NaN,NaN,NaN,7.0,NaN,NaN,2.800000e+09,COP,Mensual,Casa en ciudsd jardin,Casa independiente con posiciona en ciudad jardín y hermosos jardines. Amo la casa al\nSur se la...,Casa,Venta


## 2️⃣ **Filtrado Geográfico: Enfoque en Antioquia**

### **🎯 Decisión Estratégica**
**¿Por qué filtrar solo Antioquia?**

1. **🏙️ Homogeneidad del Mercado**: Antioquia tiene un mercado inmobiliario más homogéneo con Medellín como centro económico
2. **📊 Volumen Suficiente**: Concentra gran cantidad de transacciones para análisis estadísticamente significativo
3. **🎯 Especificidad Regional**: Los precios inmobiliarios varían significativamente entre departamentos
4. **🔍 Calidad del Análisis**: Enfoque regional permite mayor precisión en patrones de precios

### **✅ Impacto Esperado**
- Reducir variabilidad geográfica extrema
- Mejorar homogeneidad para modelos predictivos  
- Mantener volumen suficiente para análisis robusto

In [7]:
# ===============================================================
# FILTRADO GEOGRÁFICO: ANTIOQUIA CON ANÁLISIS COMPLETO
# ===============================================================

# 1. Verificar distribución por departamento (l2)
print("🌍 DISTRIBUCIÓN POR DEPARTAMENTO (TOP 10):")
print("-" * 45)
departamentos = df_original['l2'].value_counts().head(10)
total_nacional = len(df_original)

for dept, count in departamentos.items():
    porcentaje = (count / total_nacional) * 100
    print(f"   {dept:<20}: {count:>8,} ({porcentaje:>5.1f}%)")

# 2. Filtrar únicamente Antioquia  
df_antioquia = df_original[df_original['l2'] == 'Antioquia'].copy()

# 3. Resumen del filtrado geográfico
print(f"\n✅ RESULTADO DEL FILTRADO GEOGRÁFICO:")
print("=" * 45)
print(f"📊 Dataset original: {len(df_original):,} propiedades")
print(f"📊 Dataset Antioquia: {len(df_antioquia):,} propiedades") 
print(f"📈 Porcentaje conservado: {(len(df_antioquia)/len(df_original)*100):.1f}%")
print(f"📍 Enfoque regional: Departamento de Antioquia únicamente")

# 4. Análisis de ciudades en Antioquia
print(f"\n🏙️ PRINCIPALES CIUDADES EN ANTIOQUIA:")
print("-" * 40)
ciudades_antioquia = df_antioquia['l3'].value_counts().head(8)
for ciudad, count in ciudades_antioquia.items():
    porcentaje = (count / len(df_antioquia)) * 100
    print(f"   {ciudad:<20}: {count:>6,} ({porcentaje:>5.1f}%)")

# 5. Lista de barrios/sectores (l4)
print(f"\n🏘️ BARRIOS/SECTORES EN ANTIOQUIA:")
print("-" * 40)

# Limpieza básica y conteo de barrios
conteo_barrios = (
    df_antioquia['l4']
    .astype(str)
    .str.strip()
    .str.title()
    .replace('Nan', np.nan)
    .value_counts()
)

# Mostrar listado de barrios
print(f"📋 LISTADO DE BARRIOS (ORDENADO POR CANTIDAD):")
print(f"{'#':<4} {'Barrio/Sector':<35} {'Propiedades':<12} {'%':<6}")
print("-" * 65)

for i, (barrio, count) in enumerate(conteo_barrios.items(), 1):
    porcentaje = (count / len(df_antioquia)) * 100
    print(f"{i:>3}. {str(barrio):<35}: {count:>8,} ({porcentaje:>4.1f}%)")

print(f"\n✅ TOTAL: {len(conteo_barrios):,} barrios únicos")

🌍 DISTRIBUCIÓN POR DEPARTAMENTO (TOP 10):
---------------------------------------------
   Antioquia           :  341,453 ( 34.1%)
   Cundinamarca        :  208,918 ( 20.9%)
   Valle del Cauca     :  117,770 ( 11.8%)
   Atlántico           :   78,605 (  7.9%)
   Santander           :   71,737 (  7.2%)
   Caldas              :   56,296 (  5.6%)
   Norte de Santander  :   32,247 (  3.2%)
   Risaralda           :   28,505 (  2.9%)
   Quindío             :   11,753 (  1.2%)
   Bolívar             :   11,016 (  1.1%)

✅ RESULTADO DEL FILTRADO GEOGRÁFICO:
📊 Dataset original: 1,000,000 propiedades
📊 Dataset Antioquia: 341,453 propiedades
📈 Porcentaje conservado: 34.1%
📍 Enfoque regional: Departamento de Antioquia únicamente

🏙️ PRINCIPALES CIUDADES EN ANTIOQUIA:
----------------------------------------
   Medellín            : 262,856 ( 77.0%)
   Envigado            : 24,171 (  7.1%)
   Sabaneta            : 10,836 (  3.2%)
   Bello               :  8,728 (  2.6%)
   Rionegro            :  8,

## 3️⃣ **Evaluación de Calidad de Datos**

### **🎯 Objetivo**
Identificar problemas de calidad que afecten el modelo de predicción de precios.

### **🔍 Estrategia**
1. **Valores faltantes** por variable
2. **Registros duplicados** completos
3. **Rangos lógicos** en variables numéricas
4. **Consistencia** en variables categóricas

In [11]:
# ===============================================================
# EVALUACIÓN RÁPIDA DE CALIDAD DE DATOS
# ===============================================================

print("🔍 EVALUACIÓN DE CALIDAD - DATASET ANTIOQUIA")
print("=" * 50)

# 1. Información básica
print(f"📊 Dimensiones: {df_antioquia.shape}")
print(f"📊 Tipos de datos:")
print(df_antioquia.dtypes.value_counts())

# 2. Valores faltantes críticos
print(f"\n🚨 VALORES FALTANTES (Top 10):")
missing = df_antioquia.isnull().sum()
missing_pct = (missing / len(df_antioquia)) * 100
missing_top = missing[missing > 0].sort_values(ascending=False).head(10)

for col, count in missing_top.items():
    pct = missing_pct[col]
    print(f"   {col:<20}: {count:>8,} ({pct:>5.1f}%)")

# 3. Duplicados
duplicados = df_antioquia.duplicated().sum()
print(f"\n📋 REGISTROS DUPLICADOS: {duplicados:,}")

# 4. Problemas en variables clave
print(f"\n⚠️ PROBLEMAS DETECTADOS:")

# Precios inválidos
precios_invalidos = (df_antioquia['price'] <= 0).sum()
print(f"   • Precios ≤ 0: {precios_invalidos:,}")

# Coordenadas fuera de Colombia
LAT_MIN, LAT_MAX = -4.23, 15.52
LON_MIN, LON_MAX = -79.01, -66.85
coords_invalidas = (
    (df_antioquia['lat'] < LAT_MIN) | (df_antioquia['lat'] > LAT_MAX) |
    (df_antioquia['lon'] < LON_MIN) | (df_antioquia['lon'] > LON_MAX)
).sum()
print(f"   • Coordenadas fuera de Colombia: {coords_invalidas:,}")

# Habitaciones extremas
if 'bedrooms' in df_antioquia.columns:
    bedrooms_extremas = (df_antioquia['bedrooms'] > 15).sum()
    print(f"   • Dormitorios > 15: {bedrooms_extremas:,}")

print(f"\n✅ EVALUACIÓN COMPLETADA")

🔍 EVALUACIÓN DE CALIDAD - DATASET ANTIOQUIA
📊 Dimensiones: (341453, 25)
📊 Tipos de datos:
object     17
float64     8
Name: count, dtype: int64

🚨 VALORES FALTANTES (Top 10):
   l6                  :  341,453 (100.0%)
   l5                  :  341,453 (100.0%)
   surface_total       :  334,859 ( 98.1%)
   surface_covered     :  334,412 ( 97.9%)
   price_period        :  314,524 ( 92.1%)
   rooms               :  305,597 ( 89.5%)
   l4                  :  262,728 ( 76.9%)
   bedrooms            :  261,106 ( 76.5%)
   lat                 :  196,750 ( 57.6%)
   lon                 :  196,750 ( 57.6%)
   l6                  :  341,453 (100.0%)
   l5                  :  341,453 (100.0%)
   surface_total       :  334,859 ( 98.1%)
   surface_covered     :  334,412 ( 97.9%)
   price_period        :  314,524 ( 92.1%)
   rooms               :  305,597 ( 89.5%)
   l4                  :  262,728 ( 76.9%)
   bedrooms            :  261,106 ( 76.5%)
   lat                 :  196,750 ( 57.6%)
   lon  

## 4️⃣ **Limpieza de Precios Inválidos**

### **🎯 Decisión**
Eliminar registros con precio ≤ 0.

### **✅ Justificación**
Precios inválidos no sirven para predicción de precios.

In [12]:
# ===============================================================
# ELIMINAR PRECIOS INVÁLIDOS
# ===============================================================

print("🏷️ LIMPIEZA DE PRECIOS")
print("-" * 25)

inicial = len(df_antioquia)
print(f"📊 Antes: {inicial:,}")

# Eliminar precios ≤ 0
df_clean = df_antioquia[df_antioquia['price'] > 0].copy()

final = len(df_clean)
eliminados = inicial - final

print(f"✅ Después: {final:,}")
print(f"🗑️ Eliminados: {eliminados:,}")
print(f"📈 Conservado: {(final/inicial*100):.1f}%")

🏷️ LIMPIEZA DE PRECIOS
-------------------------
📊 Antes: 341,453
✅ Después: 341,373
🗑️ Eliminados: 80
📈 Conservado: 100.0%
✅ Después: 341,373
🗑️ Eliminados: 80
📈 Conservado: 100.0%


## 5️⃣ **Eliminar Coordenadas Inválidas**

### **🎯 Decisión**
Eliminar registros con coordenadas fuera de Colombia.

### **✅ Justificación**
Variables geográficas son críticas para predicción de precios.

In [13]:
# ===============================================================
# ELIMINAR COORDENADAS FUERA DE COLOMBIA
# ===============================================================

print("🌍 LIMPIEZA GEOGRÁFICA")
print("-" * 25)

# Rangos Colombia
LAT_MIN, LAT_MAX = -4.23, 15.52
LON_MIN, LON_MAX = -79.01, -66.85

inicial = len(df_clean)
print(f"📊 Antes: {inicial:,}")

# Identificar coordenadas fuera de Colombia
coords_invalidas = (
    (df_clean['lat'].notna()) & (df_clean['lon'].notna()) &
    ((df_clean['lat'] < LAT_MIN) | (df_clean['lat'] > LAT_MAX) |
     (df_clean['lon'] < LON_MIN) | (df_clean['lon'] > LON_MAX))
)

invalidas_count = coords_invalidas.sum()
print(f"🚨 Coordenadas fuera Colombia: {invalidas_count:,}")

# Eliminar registros inválidos
if invalidas_count > 0:
    df_clean = df_clean[~coords_invalidas].copy()

final = len(df_clean)
eliminados = inicial - final

print(f"✅ Después: {final:,}")
print(f"🗑️ Eliminados: {eliminados:,}")
print(f"📈 Conservado: {(final/inicial*100):.1f}%")

🌍 LIMPIEZA GEOGRÁFICA
-------------------------
📊 Antes: 341,373
🚨 Coordenadas fuera Colombia: 29
✅ Después: 341,344
🗑️ Eliminados: 29
📈 Conservado: 100.0%


## 6️⃣ **Filtros por Tipo de Propiedad**

**Decisión:** Enfocarse en apartamentos y casas únicamente

**Justificación:** Los lotes, locales comerciales y fincas tienen dinámicas de precio completamente diferentes. Concentrarse en vivienda residencial garantiza homogeneidad en el análisis

In [14]:
# ===============================================================
# FILTRAR TIPOS DE PROPIEDAD RESIDENCIAL
# ===============================================================

print("🏠 TIPOS DE PROPIEDAD")
print("-" * 25)

inicial = len(df_clean)
print(f"📊 Antes: {inicial:,}")

# Identificar tipos únicos
tipos_unicos = df_clean['property_type'].value_counts()
print(f"\n📝 Tipos de propiedad:")
for tipo, count in tipos_unicos.head(10).items():
    print(f"   {tipo}: {count:,}")

# Filtrar solo vivienda residencial
tipos_residenciales = ['Casa', 'Apartamento', 'casa', 'apartamento']
df_clean = df_clean[df_clean['property_type'].isin(tipos_residenciales)].copy()

final = len(df_clean)
eliminados = inicial - final

print(f"\n✅ Después filtro: {final:,}")
print(f"🗑️ Eliminados: {eliminados:,}")
print(f"📈 Conservado: {(final/inicial*100):.1f}%")

🏠 TIPOS DE PROPIEDAD
-------------------------
📊 Antes: 341,344

📝 Tipos de propiedad:
   Apartamento: 236,330
   Casa: 41,526
   Otro: 38,439
   Lote: 15,352
   Local comercial: 4,688
   Oficina: 3,624
   Finca: 1,142
   Depósito: 218
   Parqueadero: 25

✅ Después filtro: 277,856
🗑️ Eliminados: 63,488
📈 Conservado: 81.4%


## 7️⃣ **Extracción por Text Mining** 💎

**Decisión:** Extraer información de superficie desde descripciones usando expresiones regulares

**Justificación:** Muchas propiedades tienen superficie en el campo `description` pero no en `surface_total`. Esta es una **innovación clave** que recupera datos valiosos perdidos

In [20]:
# ===============================================================
# EXTRACCIÓN TEXT MINING - FUNCIÓN OPTIMIZADA
# ===============================================================

print("⛏️ TEXT MINING - SUPERFICIE OPTIMIZADA")
print("-" * 40)

def extraer_superficie_optimizada(descripcion):
    """Función optimizada para extraer superficie con múltiples patrones"""
    if pd.isna(descripcion):
        return None
    
    desc_lower = str(descripcion).lower()
    
    # Lista de patrones ordenados por especificidad
    patrones = [
        # Patrones principales (más específicos)
        r'(\d+(?:[.,]\d+)?)\s*(?:m2|m²|metros\s*cuadrados)',
        r'(\d+(?:[.,]\d+)?)\s*(?:mts2|mt2|metros2)',
        r'(\d+(?:[.,]\d+)?)\s*(?:metros|mts|metro)\s*(?:cuadrados?|construidos?)',
        
        # Patrones con contexto
        r'área\s*(?:de\s*|total\s*|construida\s*)?(\d+(?:[.,]\d+)?)',
        r'superficie\s*(?:de\s*|total\s*)?(\d+(?:[.,]\d+)?)',
        r'construidos?\s*(\d+(?:[.,]\d+)?)',
        r'(\d+(?:[.,]\d+)?)\s*(?:metros\s*construidos?)',
        
        # Patrones menos específicos
        r'(\d+(?:[.,]\d+)?)\s*m\s*(?:cuadrados?|construidos?)',
    ]
    
    for patron in patrones:
        matches = re.findall(patron, desc_lower)
        if matches:
            superficie_str = matches[0].replace(',', '.')
            try:
                valor = float(superficie_str)
                if 15 <= valor <= 2000:  # Rango razonable
                    return valor
            except:
                continue
    return None

# Análisis inicial
sin_superficie = df_clean['surface_total'].isna().sum()
print(f"🔍 Registros sin superficie: {sin_superficie:,}")

# Aplicar extracción optimizada
mask_sin_superficie = df_clean['surface_total'].isna()

# Limpiar columna anterior si existe
if 'surface_extracted' in df_clean.columns:
    df_clean = df_clean.drop('surface_extracted', axis=1)

# Crear nueva columna con extracciones
extracciones = df_clean.loc[mask_sin_superficie, 'description'].apply(extraer_superficie_optimizada)
df_clean.loc[mask_sin_superficie, 'surface_extracted'] = extracciones

# Estadísticas finales
extraidas = df_clean['surface_extracted'].notna().sum()
print(f"✅ Superficies extraídas: {extraidas:,}")
print(f"📈 Tasa de recuperación: {(extraidas/sin_superficie*100):.1f}%")
print(f"💎 Innovación text mining completada")

⛏️ TEXT MINING - SUPERFICIE OPTIMIZADA
----------------------------------------
🔍 Registros sin superficie: 273,094
✅ Superficies extraídas: 67,615
📈 Tasa de recuperación: 24.8%
💎 Innovación text mining completada
✅ Superficies extraídas: 67,615
📈 Tasa de recuperación: 24.8%
💎 Innovación text mining completada


In [21]:
# ===============================================================
# ANÁLISIS DE EXTRACCIÓN ADICIONAL: ROOMS, BEDROOMS, BATHROOMS
# ===============================================================

print("🔍 ANÁLISIS DE EXTRACCIÓN ADICIONAL")
print("=" * 40)

# 1. Analizar qué información falta
print("📊 ESTADO ACTUAL DE VARIABLES:")
print("-" * 35)

variables_analizar = ['rooms', 'bedrooms', 'bathrooms']
for var in variables_analizar:
    if var in df_clean.columns:
        faltantes = df_clean[var].isna().sum()
        total = len(df_clean)
        pct = (faltantes/total)*100
        print(f"   {var:<12}: {faltantes:>8,} faltantes ({pct:>5.1f}%)")
    else:
        print(f"   {var:<12}: Columna no existe")

# 2. Muestras de descripciones para análisis
print(f"\n📝 EJEMPLOS DE DESCRIPCIONES (para análisis de patrones):")
print("-" * 60)

# Muestra aleatoria de descripciones
muestra_descripciones = df_clean['description'].dropna().sample(10, random_state=42)
for i, desc in enumerate(muestra_descripciones, 1):
    print(f"\n{i:2d}. {str(desc)[:200]}...")

# 3. Búsqueda de patrones específicos
print(f"\n\n🔎 BÚSQUEDA DE PATRONES EN DESCRIPCIONES:")
print("-" * 50)

# Patrones a buscar
patrones_busqueda = {
    'habitaciones': [
        r'(\d+)\s*(?:habitacion|dormitorio|cuarto|alcoba|recamara)',
        r'(?:habitacion|dormitorio|cuarto|alcoba|recamara)\s*(\d+)',
        r'(\d+)\s*hab',
        r'(\d+)\s*habs',
    ],
    'baños': [
        r'(\d+)\s*(?:baño|baños|bathroom|wc)',
        r'(?:baño|baños|bathroom|wc)\s*(\d+)',
        r'(\d+)\s*baths?',
    ],
    'habitaciones_totales': [
        r'(\d+)\s*(?:habitaciones?|cuartos?|rooms?)',
        r'(?:habitaciones?|cuartos?|rooms?)\s*(\d+)',
    ]
}

# Análisis en muestra más grande
muestra_analisis = df_clean['description'].dropna().sample(2000, random_state=42)

for categoria, patrones in patrones_busqueda.items():
    print(f"\n🔍 {categoria.upper()}:")
    for patron in patrones:
        matches = 0
        ejemplos = []
        for desc in muestra_analisis:
            if pd.notna(desc):
                encontrado = re.search(patron, str(desc).lower())
                if encontrado:
                    matches += 1
                    if len(ejemplos) < 3:  # Guardar pocos ejemplos
                        ejemplos.append((encontrado.group(1), str(desc)[:100]))
        
        print(f"   Patrón '{patron}': {matches} matches")
        for valor, ejemplo in ejemplos:
            print(f"      → {valor} en: {ejemplo}...")

🔍 ANÁLISIS DE EXTRACCIÓN ADICIONAL
📊 ESTADO ACTUAL DE VARIABLES:
-----------------------------------
   rooms       :  244,787 faltantes ( 88.1%)
   bedrooms    :  206,506 faltantes ( 74.3%)
   bathrooms   :   25,447 faltantes (  9.2%)

📝 EJEMPLOS DE DESCRIPCIONES (para análisis de patrones):
------------------------------------------------------------

 1. Codigo Inmueble 6620 Cómodo apartamento con 3 habitaciones, sala-comedor, cocina semi-integral, zona de ropas, 1 baño completo, iluminado, con buena ventilación. Su ambiente cálido con cómodos espacio...

 2. Excelente ubicación , buenas rutas de transporte, primer piso, 1 solo ambiente, 1 baño, 1 closet, cocineta,...

 3. Codigo Inmueble 561 Casa cerca al Éxito Laureles amplios espacios con muy buenas rutas de trabajo...

 4. Codigo Inmueble 5071 Apartamento con 2 alcobas, 2 closet, sala comedor, cocina integral mixta, 2 baños cabinados, zona de ropas,  calentador a gas, red de gas, balcón, piso madera y porcelanato, área ...

 5. 

In [24]:
# ===============================================================
# FUNCIONES DE EXTRACCIÓN PARA HABITACIONES Y BAÑOS
# ===============================================================

print("🔧 CREANDO FUNCIONES DE EXTRACCIÓN")
print("-" * 40)

def extraer_bedrooms(descripcion):
    """Extrae número de dormitorios/habitaciones desde descripción"""
    if pd.isna(descripcion):
        return None
    
    desc_lower = str(descripcion).lower()
    
    # Patrones ordenados por especificidad
    patrones = [
        r'(\d+)\s*(?:habitacion|dormitorio|alcoba|recamara)(?:es)?',
        r'(?:habitacion|dormitorio|alcoba|recamara)(?:es)?\s*(\d+)',
        r'(\d+)\s*hab(?:s)?[^a-z]',
        r'(\d+)\s*dorm(?:s)?[^a-z]',
        r'(\d+)\s*bed(?:room)?s?[^a-z]',
    ]
    
    for patron in patrones:
        matches = re.findall(patron, desc_lower)
        if matches:
            try:
                valor = int(matches[0])
                if 1 <= valor <= 10:  # Rango razonable
                    return valor
            except:
                continue
    return None

def extraer_bathrooms(descripcion):
    """Extrae número de baños desde descripción"""
    if pd.isna(descripcion):
        return None
    
    desc_lower = str(descripcion).lower()
    
    # Patrones para baños
    patrones = [
        r'(\d+)\s*(?:baño|baños|bathroom|wc)s?',
        r'(?:baño|baños|bathroom|wc)s?\s*(\d+)',
        r'(\d+)\s*bath(?:s)?[^a-z]',
        r'(\d+)\s*w\.?c\.?[^a-z]',
    ]
    
    for patron in patrones:
        matches = re.findall(patron, desc_lower)
        if matches:
            try:
                valor = int(matches[0])
                if 1 <= valor <= 8:  # Rango razonable
                    return valor
            except:
                continue
    return None

def extraer_rooms(descripcion):
    """Extrae número total de habitaciones/cuartos desde descripción"""
    if pd.isna(descripcion):
        return None
    
    desc_lower = str(descripcion).lower()
    
    # Patrones para habitaciones totales
    patrones = [
        r'(\d+)\s*(?:habitaciones?|cuartos?|rooms?)(?:\s|[^a-z])',
        r'(?:habitaciones?|cuartos?|rooms?)\s*(\d+)',
        r'(\d+)\s*(?:ambientes?)',
    ]
    
    for patron in patrones:
        matches = re.findall(patron, desc_lower)
        if matches:
            try:
                valor = int(matches[0])
                if 1 <= valor <= 15:  # Rango razonable
                    return valor
            except:
                continue
    return None

print("✅ Funciones de extracción creadas")

# Probar funciones en muestra pequeña
print(f"\n🧪 PRUEBA DE FUNCIONES:")
print("-" * 25)

muestra_test = df_clean['description'].dropna().sample(1000, random_state=42)

bedrooms_extraidos = muestra_test.apply(extraer_bedrooms).notna().sum()
bathrooms_extraidos = muestra_test.apply(extraer_bathrooms).notna().sum()
rooms_extraidos = muestra_test.apply(extraer_rooms).notna().sum()

print(f"Bedrooms extraídos: {bedrooms_extraidos}/1000 ({(bedrooms_extraidos/1000*100):.1f}%)")
print(f"Bathrooms extraídos: {bathrooms_extraidos}/1000 ({(bathrooms_extraidos/1000*100):.1f}%)")
print(f"Rooms extraídos: {rooms_extraidos}/1000 ({(rooms_extraidos/1000*100):.1f}%)")

🔧 CREANDO FUNCIONES DE EXTRACCIÓN
----------------------------------------
✅ Funciones de extracción creadas

🧪 PRUEBA DE FUNCIONES:
-------------------------
Bedrooms extraídos: 657/1000 (65.7%)
Bathrooms extraídos: 538/1000 (53.8%)
Rooms extraídos: 328/1000 (32.8%)


In [25]:
# ===============================================================
# APLICAR EXTRACCIÓN AL DATASET COMPLETO
# ===============================================================

print("🚀 APLICACIÓN AL DATASET COMPLETO")
print("=" * 40)

# Aplicar solo a registros que no tienen la información
variables_extraer = {
    'bedrooms': extraer_bedrooms,
    'bathrooms': extraer_bathrooms,
    'rooms': extraer_rooms
}

for variable, funcion in variables_extraer.items():
    print(f"\n🔄 Procesando {variable}...")
    
    # Verificar si la columna existe
    if variable in df_clean.columns:
        faltantes_inicial = df_clean[variable].isna().sum()
        mask_faltantes = df_clean[variable].isna()
    else:
        print(f"   Creando nueva columna '{variable}'")
        df_clean[variable] = np.nan
        faltantes_inicial = len(df_clean)
        mask_faltantes = df_clean[variable].isna()
    
    print(f"   Registros sin {variable}: {faltantes_inicial:,}")
    
    # Aplicar extracción
    extracciones = df_clean.loc[mask_faltantes, 'description'].apply(funcion)
    df_clean.loc[mask_faltantes, f'{variable}_extracted'] = extracciones
    
    # Estadísticas
    extraidos = df_clean[f'{variable}_extracted'].notna().sum()
    tasa_recuperacion = (extraidos / faltantes_inicial * 100) if faltantes_inicial > 0 else 0
    
    print(f"   ✅ Extraídos: {extraidos:,}")
    print(f"   📈 Tasa recuperación: {tasa_recuperacion:.1f}%")

print(f"\n💎 EXTRACCIÓN ADICIONAL COMPLETADA")
print("=" * 40)

# Resumen final de extracciones
print(f"📊 RESUMEN DE TODAS LAS EXTRACCIONES:")
print("-" * 35)
print(f"   Surface: {df_clean['surface_extracted'].notna().sum():,} extraídas")
print(f"   Bedrooms: {df_clean['bedrooms_extracted'].notna().sum():,} extraídas")
print(f"   Bathrooms: {df_clean['bathrooms_extracted'].notna().sum():,} extraídas")
print(f"   Rooms: {df_clean['rooms_extracted'].notna().sum():,} extraídas")

total_extracciones = (
    df_clean['surface_extracted'].notna().sum() +
    df_clean['bedrooms_extracted'].notna().sum() +
    df_clean['bathrooms_extracted'].notna().sum() +
    df_clean['rooms_extracted'].notna().sum()
)

print(f"\n🎯 TOTAL DATOS RECUPERADOS: {total_extracciones:,}")
print(f"🏆 Text Mining: INNOVACIÓN COMPLETA")

🚀 APLICACIÓN AL DATASET COMPLETO

🔄 Procesando bedrooms...
   Registros sin bedrooms: 206,506
   ✅ Extraídos: 144,970
   📈 Tasa recuperación: 70.2%

🔄 Procesando bathrooms...
   Registros sin bathrooms: 25,447
   ✅ Extraídos: 144,970
   📈 Tasa recuperación: 70.2%

🔄 Procesando bathrooms...
   Registros sin bathrooms: 25,447
   ✅ Extraídos: 12,824
   📈 Tasa recuperación: 50.4%

🔄 Procesando rooms...
   Registros sin rooms: 244,787
   ✅ Extraídos: 12,824
   📈 Tasa recuperación: 50.4%

🔄 Procesando rooms...
   Registros sin rooms: 244,787
   ✅ Extraídos: 90,968
   📈 Tasa recuperación: 37.2%

💎 EXTRACCIÓN ADICIONAL COMPLETADA
📊 RESUMEN DE TODAS LAS EXTRACCIONES:
-----------------------------------
   Surface: 67,615 extraídas
   Bedrooms: 144,970 extraídas
   Bathrooms: 12,824 extraídas
   Rooms: 90,968 extraídas

🎯 TOTAL DATOS RECUPERADOS: 316,377
🏆 Text Mining: INNOVACIÓN COMPLETA
   ✅ Extraídos: 90,968
   📈 Tasa recuperación: 37.2%

💎 EXTRACCIÓN ADICIONAL COMPLETADA
📊 RESUMEN DE TODAS L

## 8️⃣ **Integración de Datos Extraídos**

**Decisión:** Consolidar datos originales con extracciones de text mining

**Justificación:** Crear variables finales que combinen datos originales con extracciones para maximizar completitud sin perder información original

In [26]:
# ===============================================================
# INTEGRACIÓN DE DATOS ORIGINALES CON EXTRACCIONES
# ===============================================================

print("🔗 INTEGRACIÓN DE DATOS")
print("=" * 30)

# Estrategia: Priorizar datos originales, completar con extracciones
variables_integrar = {
    'surface_total': 'surface_extracted',
    'bedrooms': 'bedrooms_extracted', 
    'bathrooms': 'bathrooms_extracted',
    'rooms': 'rooms_extracted'
}

for original, extraida in variables_integrar.items():
    if extraida in df_clean.columns:
        # Estado antes de integración
        antes_faltantes = df_clean[original].isna().sum()
        
        # Integrar: usar original si existe, sino usar extraída
        df_clean[f'{original}_final'] = df_clean[original].fillna(df_clean[extraida])
        
        # Estado después de integración
        despues_faltantes = df_clean[f'{original}_final'].isna().sum()
        completados = antes_faltantes - despues_faltantes
        
        print(f"\n📊 {original.upper()}:")
        print(f"   Antes: {antes_faltantes:,} faltantes")
        print(f"   Después: {despues_faltantes:,} faltantes")
        print(f"   ✅ Completados: {completados:,}")
        if antes_faltantes > 0:
            print(f"   📈 Mejora: {(completados/antes_faltantes*100):.1f}%")

print(f"\n🎯 RESUMEN DE INTEGRACIÓN:")
print("-" * 30)

# Calcular total de completaciones
total_completaciones = 0
for original, extraida in variables_integrar.items():
    if extraida in df_clean.columns:
        antes = df_clean[original].isna().sum()
        despues = df_clean[f'{original}_final'].isna().sum()
        total_completaciones += (antes - despues)

print(f"📈 Total valores completados: {total_completaciones:,}")
print(f"💎 Integración exitosa completada")

🔗 INTEGRACIÓN DE DATOS

📊 SURFACE_TOTAL:
   Antes: 273,094 faltantes
   Después: 205,479 faltantes
   ✅ Completados: 67,615
   📈 Mejora: 24.8%

📊 BEDROOMS:
   Antes: 206,506 faltantes
   Después: 61,536 faltantes
   ✅ Completados: 144,970
   📈 Mejora: 70.2%

📊 BATHROOMS:
   Antes: 25,447 faltantes
   Después: 12,623 faltantes
   ✅ Completados: 12,824
   📈 Mejora: 50.4%

📊 ROOMS:
   Antes: 244,787 faltantes
   Después: 153,819 faltantes
   ✅ Completados: 90,968
   📈 Mejora: 37.2%

🎯 RESUMEN DE INTEGRACIÓN:
------------------------------
📈 Total valores completados: 316,377
💎 Integración exitosa completada


In [28]:
# ===============================================================
# VALIDACIÓN DETALLADA DE LA INTEGRACIÓN 
# ===============================================================

print("🔍 VALIDACIÓN DETALLADA DE INTEGRACIÓN")
print("=" * 50)

# Verificar que la lógica de integración sea correcta
variables_validar = ['surface_total', 'bedrooms', 'bathrooms', 'rooms']

for var in variables_validar:
    var_final = f'{var}_final'
    var_extracted = f'{var}_extracted'
    
    if var_final in df_clean.columns:
        print(f"\n🔎 VALIDANDO {var.upper()}:")
        print("-" * 25)
        
        # 1. Verificar que los valores originales se mantienen
        original_disponible = df_clean[var].notna()
        valores_originales_mantenidos = (
            df_clean.loc[original_disponible, var] == 
            df_clean.loc[original_disponible, var_final]
        ).all()
        
        print(f"   ✅ Valores originales mantenidos: {valores_originales_mantenidos}")
        
        # 2. Verificar que extracciones van donde falta original
        if var_extracted in df_clean.columns:
            sin_original = df_clean[var].isna()
            con_extraido = df_clean[var_extracted].notna()
            
            # Casos donde NO había original PERO SÍ hay extraído
            casos_llenados = sin_original & con_extraido
            valores_extraidos_correctos = (
                df_clean.loc[casos_llenados, var_extracted] == 
                df_clean.loc[casos_llenados, var_final]
            ).all()
            
            print(f"   ✅ Extracciones van donde falta: {valores_extraidos_correctos}")
            print(f"   📊 Casos completados: {casos_llenados.sum():,}")
            
            # 3. Muestra de casos específicos
            if casos_llenados.sum() > 0:
                muestra = df_clean.loc[casos_llenados, [var, var_extracted, var_final]].head(3)
                print(f"   📋 Muestra casos completados:")
                for idx, row in muestra.iterrows():
                    original_val = row[var]
                    extracted_val = row[var_extracted] 
                    final_val = row[var_final]
                    print(f"      Original: {original_val} → Extraído: {extracted_val} → Final: {final_val}")
        
        # 4. Verificar que NO se sobrescriben valores originales
        if var_extracted in df_clean.columns:
            con_original = df_clean[var].notna()
            casos_con_ambos = con_original & df_clean[var_extracted].notna()
            
            if casos_con_ambos.sum() > 0:
                sobrescrituras = (
                    df_clean.loc[casos_con_ambos, var] != 
                    df_clean.loc[casos_con_ambos, var_final]
                ).sum()
                print(f"   🚨 Sobrescrituras incorrectas: {sobrescrituras}")
                
                if sobrescrituras > 0:
                    print("   ❌ ERROR: Valores originales fueron sobrescritos!")
                else:
                    print("   ✅ No hay sobrescrituras incorrectas")

# 5. Resumen final de validación
print(f"\n🎯 RESUMEN DE VALIDACIÓN:")
print("-" * 30)

for var in variables_validar:
    var_final = f'{var}_final'
    if var_final in df_clean.columns:
        original_count = df_clean[var].notna().sum()
        final_count = df_clean[var_final].notna().sum()
        mejora = final_count - original_count
        print(f"   {var:<15}: {original_count:>6,} → {final_count:>6,} (+{mejora:>5,})")

print(f"\n✅ VALIDACIÓN DE INTEGRACIÓN COMPLETADA")

🔍 VALIDACIÓN DETALLADA DE INTEGRACIÓN

🔎 VALIDANDO SURFACE_TOTAL:
-------------------------
   ✅ Valores originales mantenidos: True

🔎 VALIDANDO BEDROOMS:
-------------------------
   ✅ Valores originales mantenidos: True
   ✅ Extracciones van donde falta: True
   📊 Casos completados: 144,970
   📋 Muestra casos completados:
      Original: nan → Extraído: 7.0 → Final: 7.0
      Original: nan → Extraído: 5.0 → Final: 5.0
      Original: nan → Extraído: 5.0 → Final: 5.0

🔎 VALIDANDO BATHROOMS:
-------------------------
   ✅ Valores originales mantenidos: True
   ✅ Extracciones van donde falta: True
   📊 Casos completados: 12,824
   📋 Muestra casos completados:
      Original: nan → Extraído: 3.0 → Final: 3.0
      Original: nan → Extraído: 3.0 → Final: 3.0
      Original: nan → Extraído: 2.0 → Final: 2.0

🔎 VALIDANDO ROOMS:
-------------------------
   ✅ Valores originales mantenidos: True
   ✅ Extracciones van donde falta: True
   📊 Casos completados: 90,968
   📋 Muestra casos completa

## 9️⃣ **Tratamiento de Valores Faltantes**

**Objetivo:** Imputar valores faltantes con estrategias estadísticas antes de la imputación avanzada de superficie

**Justificación:** Completar variables críticas como coordenadas y ubicación para maximizar la calidad del dataset antes del modelado ML de superficie

In [29]:
# ===============================================================
# ANÁLISIS DE VALORES FALTANTES RESTANTES
# ===============================================================

print("🔍 ANÁLISIS DE VALORES FALTANTES RESTANTES")
print("=" * 50)

# Evaluar estado actual después de text mining e integración
variables_criticas = ['lat', 'lon', 'l4', 'surface_total_final', 'bedrooms_final', 'bathrooms_final']

print("📊 ESTADO ACTUAL DE VARIABLES CRÍTICAS:")
print("-" * 45)

for var in variables_criticas:
    if var in df_clean.columns:
        faltantes = df_clean[var].isna().sum()
        total = len(df_clean)
        pct = (faltantes / total) * 100
        print(f"   {var:<20}: {faltantes:>8,} faltantes ({pct:>5.1f}%)")

# Identificar variables para imputación simple
print(f"\n🎯 ESTRATEGIAS DE IMPUTACIÓN:")
print("-" * 35)

# 1. Coordenadas faltantes - imputar por centroide de barrio
coords_faltantes = df_clean[['lat', 'lon']].isna().any(axis=1).sum()
barrios_disponibles = df_clean['l4'].notna().sum()
print(f"   Coordenadas: {coords_faltantes:,} faltantes")
print(f"   Barrios disponibles: {barrios_disponibles:,}")

# 2. Barrios faltantes - estrategia compleja (mantener para análisis)
barrios_faltantes = df_clean['l4'].isna().sum()
print(f"   Barrios: {barrios_faltantes:,} faltantes")

print(f"\n✅ ANÁLISIS COMPLETADO - LISTO PARA IMPUTACIÓN")

🔍 ANÁLISIS DE VALORES FALTANTES RESTANTES
📊 ESTADO ACTUAL DE VARIABLES CRÍTICAS:
---------------------------------------------
   lat                 :  163,274 faltantes ( 58.8%)
   lon                 :  163,274 faltantes ( 58.8%)
   l4                  :  211,876 faltantes ( 76.3%)
   surface_total_final :  205,479 faltantes ( 74.0%)
   bedrooms_final      :   61,536 faltantes ( 22.1%)
   bathrooms_final     :   12,623 faltantes (  4.5%)

🎯 ESTRATEGIAS DE IMPUTACIÓN:
-----------------------------------
   Coordenadas: 163,274 faltantes
   Barrios disponibles: 65,980
   Barrios: 211,876 faltantes

✅ ANÁLISIS COMPLETADO - LISTO PARA IMPUTACIÓN


## 🔟 **Extracción de Ubicación por Text Mining** 🌍

**Objetivo:** Extraer información de ubicación (barrios, sectores, zonas) desde las descripciones

**Justificación:** Con 211,876 registros sin barrio (l4) pero con descripciones disponibles, esta estrategia puede recuperar información geográfica valiosa que está "escondida" en el texto libre.

**Estrategia:** Usar patrones específicos para identificar menciones de barrios, sectores y zonas conocidas de Antioquia

In [35]:
# ===============================================================
# ANÁLISIS DE DESCRIPCIONES PARA EXTRACCIÓN DE UBICACIÓN
# ===============================================================

print("🌍 EXTRACCIÓN DE UBICACIÓN POR TEXT MINING")
print("=" * 50)

# 1. Estado actual de ubicaciones
print("📊 ESTADO ACTUAL DE UBICACIÓN:")
print("-" * 35)

total_registros = len(df_clean)
con_barrio = df_clean['l4'].notna().sum()
sin_barrio = df_clean['l4'].isna().sum()
con_descripcion = df_clean['description'].notna().sum()

print(f"Total registros: {total_registros:,}")
print(f"Con barrio (l4): {con_barrio:,} ({(con_barrio/total_registros*100):.1f}%)")
print(f"Sin barrio (l4): {sin_barrio:,} ({(sin_barrio/total_registros*100):.1f}%)")
print(f"Con descripción: {con_descripcion:,} ({(con_descripcion/total_registros*100):.1f}%)")

# 2. Candidatos para extracción: sin barrio PERO con descripción
candidatos_ubicacion = (df_clean['l4'].isna() & df_clean['description'].notna()).sum()
print(f"\n🎯 Candidatos para extracción: {candidatos_ubicacion:,}")
print(f"   (Sin barrio PERO con descripción)")

# 3. Obtener barrios conocidos para crear patrones
print(f"\n📋 BARRIOS CONOCIDOS EN EL DATASET:")
print("-" * 35)

barrios_conocidos = df_clean['l4'].dropna().unique()
print(f"Total barrios únicos: {len(barrios_conocidos):,}")

# Mostrar los más comunes
top_barrios = df_clean['l4'].value_counts().head(20)
print(f"\nTop 20 barrios más comunes:")
for i, (barrio, count) in enumerate(top_barrios.items(), 1):
    print(f"  {i:2d}. {barrio:<30}: {count:>4,} propiedades")

🌍 EXTRACCIÓN DE UBICACIÓN POR TEXT MINING
📊 ESTADO ACTUAL DE UBICACIÓN:
-----------------------------------
Total registros: 277,856
Con barrio (l4): 65,980 (23.7%)
Sin barrio (l4): 211,876 (76.3%)
Con descripción: 277,597 (99.9%)

🎯 Candidatos para extracción: 211,706
   (Sin barrio PERO con descripción)

📋 BARRIOS CONOCIDOS EN EL DATASET:
-----------------------------------
Total barrios únicos: 21

Top 20 barrios más comunes:
   1. El Poblado                    : 21,896 propiedades
   2. Laureles                      : 11,039 propiedades
   3. Belén                         : 8,897 propiedades
   4. La América                    : 5,747 propiedades
   5. Robledo                       : 3,487 propiedades
   6. Buenos Aires                  : 3,306 propiedades
   7. Candelaria                    : 3,291 propiedades
   8. San Javier                    : 1,177 propiedades
   9. Guayabal                      : 1,036 propiedades
  10. Castilla                      : 1,010 propiedades
  11.

In [38]:
# ===============================================================
# ANÁLISIS DE PATRONES EN DESCRIPCIONES
# ===============================================================

print("🔍 ANÁLISIS DE PATRONES DE UBICACIÓN EN DESCRIPCIONES")
print("=" * 60)

# Muestra de descripciones de registros SIN barrio para análisis
sin_barrio_mask = df_clean['l4'].isna() & df_clean['description'].notna()
muestra_sin_barrio = df_clean[sin_barrio_mask]['description'].sample(15, random_state=42)

print("📝 MUESTRA DE DESCRIPCIONES SIN BARRIO:")
print("-" * 45)
for i, desc in enumerate(muestra_sin_barrio, 1):
    desc_corta = str(desc)[:120] + "..." if len(str(desc)) > 120 else str(desc)
    print(f"\n{i:2d}. {desc_corta}")

# Búsqueda específica de barrios conocidos en descripciones
print(f"\n\n🎯 BÚSQUEDA DE BARRIOS CONOCIDOS EN DESCRIPCIONES:")
print("-" * 55)

# ANÁLISIS DE CIUDADES DISPONIBLES EN EL DATASET
print(f"\n🏙️ ANÁLISIS DE CIUDADES DISPONIBLES:")
print("-" * 40)

# Obtener ciudades (l3) del dataset actual
ciudades_disponibles = df_clean['l3'].dropna().unique()
conteo_ciudades = df_clean['l3'].value_counts()

print(f"Total ciudades únicas: {len(ciudades_disponibles)}")
print(f"\nCiudades ordenadas por cantidad de propiedades:")
for i, (ciudad, count) in enumerate(conteo_ciudades.items(), 1):
    pct = (count / len(df_clean)) * 100
    print(f"  {i:2d}. {ciudad:<25}: {count:>6,} ({pct:>5.1f}%)")

# Crear lista de ciudades para buscar (basada en el dataset real)
ciudades_buscar = list(ciudades_disponibles)
print(f"\n🎯 Lista de ciudades para extracción: {len(ciudades_buscar)} ciudades")

# Buscar cada ciudad en descripciones SIN ciudad asignada
sin_ciudad_mask = df_clean['l3'].isna() & df_clean['description'].notna()
registros_sin_ciudad = sin_ciudad_mask.sum()
print(f"📊 Registros sin ciudad (l3) pero con descripción: {registros_sin_ciudad:,}")

# Analizar una muestra más pequeña para ciudades
muestra_sin_ciudad = df_clean[sin_ciudad_mask]['description'].sample(min(50, registros_sin_ciudad), random_state=42)

encontrados_ciudades = {}
for ciudad in ciudades_buscar:
    # Patrón flexible para encontrar la ciudad
    patron = rf'\b{re.escape(ciudad.lower())}\b'
    
    # Buscar en descripciones (convertir a minúsculas)
    matches = 0
    ejemplos = []
    
    for desc in muestra_sin_ciudad:
        if pd.notna(desc):
            desc_lower = str(desc).lower()
            if re.search(patron, desc_lower):
                matches += 1
                if len(ejemplos) < 2:  # Máximo 2 ejemplos
                    ejemplos.append(str(desc)[:100] + "...")
    
    if matches > 0:
        encontrados_ciudades[ciudad] = {'matches': matches, 'ejemplos': ejemplos}

# Mostrar resultados de ciudades
print(f"\nCiudades encontradas en muestra de {len(muestra_sin_ciudad)} descripciones:")
for ciudad, info in encontrados_ciudades.items():
    print(f"\n• {ciudad}: {info['matches']} matches")
    for ejemplo in info['ejemplos']:
        print(f"  → {ejemplo}")

print(f"\n✅ Análisis de ciudades completado")
print(f"📊 Ciudades detectadas: {len(encontrados_ciudades)} de {len(ciudades_buscar)} buscadas")

🔍 ANÁLISIS DE PATRONES DE UBICACIÓN EN DESCRIPCIONES
📝 MUESTRA DE DESCRIPCIONES SIN BARRIO:
---------------------------------------------

 1. Codigo Inmueble 505512 Excelente casa de dos niveles, la cual cuenta con 6 habitaciones. 5 baños, parqueadero doble, amp...

 2. Apto 501: 
80 mt2
Estrato 3
Dos años de antiguedad
Predial (semestral): $26.800
No paga admon 
- Alcoba principal con ba...

 3. Apartamento para arriendo en Campus Reservado 
Cuenta con: 2 habitaciones, 2 baños, 1 closet, 1 Vestier, calentador, sal...

 4. Codigo Inmueble 504708 Acogedora casa ubicado en un sector muy residencial con cómodos espacios buena distribución y exc...

 5. Codigo Inmueble 340 Apartamento para arriendo o Venta, tercer piso, parqueadero cubierto, citofonía, cerca a unicentro, ...

 6. Hermoso apartamento de 114 M2, tres habitaciones, tres baños, sala comedor, cocina integral abierta, zona de ropas, vist...

 7. Codigo Inmueble 718 Arrienda apartamento 75 m2 , 3 alcobas , 2 closet, 1 baño cabin

In [39]:
# ===============================================================
# FUNCIONES DE EXTRACCIÓN: CIUDAD Y BARRIO
# ===============================================================

print("🏗️ CREANDO FUNCIONES DE EXTRACCIÓN DE UBICACIÓN")
print("=" * 55)

def extraer_ciudad(descripcion):
    """Extrae ciudad desde descripción usando ciudades conocidas del dataset"""
    if pd.isna(descripcion):
        return None
    
    desc_lower = str(descripcion).lower()
    
    # Lista de ciudades del dataset (en orden de frecuencia)
    ciudades_buscar = [
        'medellín', 'medellin', 'envigado', 'itagüí', 'itagui', 'sabaneta', 
        'bello', 'copacabana', 'la estrella', 'estrella', 'caldas', 
        'girardota', 'barbosa', 'rionegro'
    ]
    
    # Buscar cada ciudad con patrones específicos
    for ciudad in ciudades_buscar:
        # Patrón que busca la ciudad como palabra completa
        patron = rf'\b{re.escape(ciudad)}\b'
        
        if re.search(patron, desc_lower):
            # Mapear a nombre estándar del dataset
            if ciudad in ['medellin', 'medellín']:
                return 'Medellín'
            elif ciudad in ['itagui', 'itagüí']:
                return 'Itagüí'
            elif ciudad in ['estrella', 'la estrella']:
                return 'La Estrella'
            else:
                return ciudad.title()
    
    return None

def extraer_barrio(descripcion):
    """Extrae barrio desde descripción usando barrios conocidos del dataset"""
    if pd.isna(descripcion):
        return None
    
    desc_lower = str(descripcion).lower()
    
    # Lista de barrios conocidos del dataset (solo barrios reales, no ciudades)
    barrios_buscar = [
        'el poblado', 'poblado', 'laureles', 'belén', 'belen', 'la américa', 'america',
        'robledo', 'buenos aires', 'candelaria', 'san javier', 'guayabal', 'castilla',
        'san cristóbal', 'san cristobal', 'aranjuez', 'altavista', 'villa hermosa',
        'santa elena', 'san antonio de prado', 'manrique', 'doce de octubre', 
        'santa cruz', 'popular'
    ]
    
    # Buscar cada barrio con patrones específicos
    for barrio in barrios_buscar:
        # Patrón que busca el barrio como palabra completa
        patron = rf'\b{re.escape(barrio)}\b'
        
        if re.search(patron, desc_lower):
            # Mapear a nombre estándar del dataset
            if barrio in ['poblado', 'el poblado']:
                return 'El Poblado'
            elif barrio in ['belen', 'belén']:
                return 'Belén'
            elif barrio in ['america', 'la américa']:
                return 'La América'
            elif barrio in ['san cristobal', 'san cristóbal']:
                return 'San Cristóbal'
            else:
                return barrio.title()
    
    return None

print("✅ Funciones de extracción creadas")

# Probar funciones en muestra pequeña
print(f"\n🧪 PRUEBA DE FUNCIONES DE UBICACIÓN:")
print("-" * 40)

# Muestra para probar
muestra_test_ubicacion = df_clean['description'].dropna().sample(1000, random_state=42)

ciudades_extraidas = muestra_test_ubicacion.apply(extraer_ciudad).notna().sum()
barrios_extraidos = muestra_test_ubicacion.apply(extraer_barrio).notna().sum()

print(f"Ciudades extraídas: {ciudades_extraidas}/1000 ({(ciudades_extraidas/1000*100):.1f}%)")
print(f"Barrios extraídos: {barrios_extraidos}/1000 ({(barrios_extraidos/1000*100):.1f}%)")

# Mostrar ejemplos de extracciones exitosas
print(f"\n📋 EJEMPLOS DE EXTRACCIONES EXITOSAS:")
print("-" * 40)

ejemplos_ciudad = muestra_test_ubicacion[muestra_test_ubicacion.apply(extraer_ciudad).notna()].head(3)
ejemplos_barrio = muestra_test_ubicacion[muestra_test_ubicacion.apply(extraer_barrio).notna()].head(3)

print("🏙️ CIUDADES:")
for i, desc in enumerate(ejemplos_ciudad, 1):
    ciudad_encontrada = extraer_ciudad(desc)
    print(f"  {i}. Ciudad: {ciudad_encontrada}")
    print(f"     Descripción: {str(desc)[:80]}...")

print("\n🏘️ BARRIOS:")
for i, desc in enumerate(ejemplos_barrio, 1):
    barrio_encontrado = extraer_barrio(desc)
    print(f"  {i}. Barrio: {barrio_encontrado}")
    print(f"     Descripción: {str(desc)[:80]}...")

print(f"\n✅ PRUEBAS COMPLETADAS")

🏗️ CREANDO FUNCIONES DE EXTRACCIÓN DE UBICACIÓN
✅ Funciones de extracción creadas

🧪 PRUEBA DE FUNCIONES DE UBICACIÓN:
----------------------------------------
Ciudades extraídas: 233/1000 (23.3%)
Barrios extraídos: 183/1000 (18.3%)

📋 EJEMPLOS DE EXTRACCIONES EXITOSAS:
----------------------------------------
Ciudades extraídas: 233/1000 (23.3%)
Barrios extraídos: 183/1000 (18.3%)

📋 EJEMPLOS DE EXTRACCIONES EXITOSAS:
----------------------------------------
🏙️ CIUDADES:
  1. Ciudad: Medellín
     Descripción: ¿Estás buscando pent-house, con excelente ubicación iluminacion natural y un amp...
  2. Ciudad: Itagüí
     Descripción: 622-14137 Apartamento en arriendo ubicado en Itagüí sector Suramérica.Excelentes...
  3. Ciudad: La Estrella
     Descripción: <b>Suramerica, La Estrella, arriendo apartamento</b><br><br>Arriendo apartamento...

🏘️ BARRIOS:
  1. Barrio: Laureles
     Descripción: Codigo Inmueble 561 Casa cerca al Éxito Laureles amplios espacios con muy buenas...
  2. Barrio: 

In [40]:
# ===============================================================
# APLICAR EXTRACCIÓN DE UBICACIÓN AL DATASET COMPLETO
# ===============================================================

print("🚀 APLICACIÓN AL DATASET COMPLETO - UBICACIÓN")
print("=" * 50)

# PASO 1: EXTRACCIÓN DE CIUDADES (l3)
print("\n🏙️ EXTRACCIÓN DE CIUDADES:")
print("-" * 35)

# Estado inicial de ciudades
ciudades_faltantes_inicial = df_clean['l3'].isna().sum()
print(f"Ciudades faltantes antes: {ciudades_faltantes_inicial:,}")

# Aplicar extracción solo a registros sin ciudad pero con descripción
mask_sin_ciudad = df_clean['l3'].isna() & df_clean['description'].notna()
candidatos_ciudad = mask_sin_ciudad.sum()
print(f"Candidatos para extracción: {candidatos_ciudad:,}")

# Extraer ciudades
extracciones_ciudad = df_clean.loc[mask_sin_ciudad, 'description'].apply(extraer_ciudad)
df_clean.loc[mask_sin_ciudad, 'l3_extracted'] = extracciones_ciudad

# Estadísticas de ciudades
ciudades_extraidas = df_clean['l3_extracted'].notna().sum()
tasa_ciudad = (ciudades_extraidas / candidatos_ciudad * 100) if candidatos_ciudad > 0 else 0

print(f"✅ Ciudades extraídas: {ciudades_extraidas:,}")
print(f"📈 Tasa de éxito: {tasa_ciudad:.1f}%")

# PASO 2: EXTRACCIÓN DE BARRIOS (l4)
print(f"\n🏘️ EXTRACCIÓN DE BARRIOS:")
print("-" * 35)

# Estado inicial de barrios
barrios_faltantes_inicial = df_clean['l4'].isna().sum()
print(f"Barrios faltantes antes: {barrios_faltantes_inicial:,}")

# Aplicar extracción solo a registros sin barrio pero con descripción
mask_sin_barrio = df_clean['l4'].isna() & df_clean['description'].notna()
candidatos_barrio = mask_sin_barrio.sum()
print(f"Candidatos para extracción: {candidatos_barrio:,}")

# Extraer barrios
extracciones_barrio = df_clean.loc[mask_sin_barrio, 'description'].apply(extraer_barrio)
df_clean.loc[mask_sin_barrio, 'l4_extracted'] = extracciones_barrio

# Estadísticas de barrios
barrios_extraidos = df_clean['l4_extracted'].notna().sum()
tasa_barrio = (barrios_extraidos / candidatos_barrio * 100) if candidatos_barrio > 0 else 0

print(f"✅ Barrios extraídos: {barrios_extraidos:,}")
print(f"📈 Tasa de éxito: {tasa_barrio:.1f}%")

# PASO 3: INTEGRACIÓN DE UBICACIONES
print(f"\n🔗 INTEGRACIÓN DE UBICACIONES:")
print("-" * 35)

# Integrar ciudades: usar original si existe, sino usar extraída
df_clean['l3_final'] = df_clean['l3'].fillna(df_clean['l3_extracted'])

# Integrar barrios: usar original si existe, sino usar extraída  
df_clean['l4_final'] = df_clean['l4'].fillna(df_clean['l4_extracted'])

# Estadísticas finales
ciudades_final = df_clean['l3_final'].notna().sum()
barrios_final = df_clean['l4_final'].notna().sum()

ciudades_completadas = ciudades_final - (len(df_clean) - ciudades_faltantes_inicial)
barrios_completados = barrios_final - (len(df_clean) - barrios_faltantes_inicial)

print(f"📊 RESULTADOS FINALES:")
print("-" * 25)
print(f"Ciudades completadas: {ciudades_completadas:,}")
print(f"Barrios completados: {barrios_completados:,}")

# Mostrar distribución de ciudades y barrios extraídos
print(f"\n📋 DISTRIBUCIÓN DE EXTRACCIONES:")
print("-" * 35)

if 'l3_extracted' in df_clean.columns:
    print("🏙️ Ciudades extraídas:")
    ciudades_extraidas_dist = df_clean['l3_extracted'].value_counts().head(10)
    for ciudad, count in ciudades_extraidas_dist.items():
        print(f"   {ciudad:<20}: {count:>4,}")

if 'l4_extracted' in df_clean.columns:
    print("\n🏘️ Barrios extraídos:")
    barrios_extraidos_dist = df_clean['l4_extracted'].value_counts().head(10)
    for barrio, count in barrios_extraidos_dist.items():
        print(f"   {barrio:<20}: {count:>4,}")

print(f"\n🎯 TOTAL UBICACIONES RECUPERADAS: {ciudades_extraidas + barrios_extraidos:,}")
print(f"🏆 Extracción de ubicación: COMPLETADA")

🚀 APLICACIÓN AL DATASET COMPLETO - UBICACIÓN

🏙️ EXTRACCIÓN DE CIUDADES:
-----------------------------------
Ciudades faltantes antes: 3,689
Candidatos para extracción: 3,649
✅ Ciudades extraídas: 1,019
📈 Tasa de éxito: 27.9%

🏘️ EXTRACCIÓN DE BARRIOS:
-----------------------------------
Barrios faltantes antes: 211,876
Candidatos para extracción: 211,706
✅ Ciudades extraídas: 1,019
📈 Tasa de éxito: 27.9%

🏘️ EXTRACCIÓN DE BARRIOS:
-----------------------------------
Barrios faltantes antes: 211,876
Candidatos para extracción: 211,706
✅ Barrios extraídos: 25,775
📈 Tasa de éxito: 12.2%

🔗 INTEGRACIÓN DE UBICACIONES:
-----------------------------------
📊 RESULTADOS FINALES:
-------------------------
Ciudades completadas: 1,019
Barrios completados: 25,775

📋 DISTRIBUCIÓN DE EXTRACCIONES:
-----------------------------------
🏙️ Ciudades extraídas:
   Rionegro            :  516
   Medellín            :  445
   Envigado            :   25
   Sabaneta            :   15
   La Estrella         : 

In [41]:
# ===============================================================
# VALIDACIÓN Y RESUMEN DE EXTRACCIÓN DE UBICACIÓN
# ===============================================================

print("🔍 VALIDACIÓN DE EXTRACCIÓN DE UBICACIÓN")
print("=" * 50)

# 1. Comparar estado antes vs después
print("📊 COMPARACIÓN ANTES vs DESPUÉS:")
print("-" * 35)

# Ciudades
ciudad_antes = df_clean['l3'].notna().sum()
ciudad_despues = df_clean['l3_final'].notna().sum()
mejora_ciudad = ciudad_despues - ciudad_antes

print(f"🏙️ CIUDADES:")
print(f"   Antes: {ciudad_antes:,}")
print(f"   Después: {ciudad_despues:,}")
print(f"   Mejora: +{mejora_ciudad:,} ({(mejora_ciudad/len(df_clean)*100):.1f}%)")

# Barrios  
barrio_antes = df_clean['l4'].notna().sum()
barrio_despues = df_clean['l4_final'].notna().sum()
mejora_barrio = barrio_despues - barrio_antes

print(f"\n🏘️ BARRIOS:")
print(f"   Antes: {barrio_antes:,}")
print(f"   Después: {barrio_despues:,}")
print(f"   Mejora: +{mejora_barrio:,} ({(mejora_barrio/len(df_clean)*100):.1f}%)")

# 2. Impacto en coordenadas faltantes
print(f"\n🗺️ IMPACTO EN COORDENADAS:")
print("-" * 30)

# Recalcular potencial de imputación de coordenadas
coords_faltantes = df_clean[['lat', 'lon']].isna().any(axis=1).sum()
barrios_disponibles_ahora = df_clean['l4_final'].notna().sum()

# Casos donde ahora PODRÍAMOS imputar coordenadas
sin_coords_con_barrio_ahora = (
    df_clean[['lat', 'lon']].isna().any(axis=1) & 
    df_clean['l4_final'].notna()
).sum()

print(f"Coordenadas faltantes: {coords_faltantes:,}")
print(f"Barrios disponibles ahora: {barrios_disponibles_ahora:,}")
print(f"🎯 Casos para imputar coords: {sin_coords_con_barrio_ahora:,}")

# 3. Muestra de casos exitosos
print(f"\n📋 MUESTRA DE CASOS EXITOSOS:")
print("-" * 35)

# Casos donde se recuperó barrio
casos_barrio_recuperado = df_clean[
    df_clean['l4'].isna() & 
    df_clean['l4_extracted'].notna()
][['description', 'l4_extracted']].head(5)

print("🏘️ Barrios recuperados:")
for idx, row in casos_barrio_recuperado.iterrows():
    desc = str(row['description'])[:80] + "..."
    barrio = row['l4_extracted']
    print(f"   → {barrio}: {desc}")

# Casos donde se recuperó ciudad
casos_ciudad_recuperada = df_clean[
    df_clean['l3'].isna() & 
    df_clean['l3_extracted'].notna()
][['description', 'l3_extracted']].head(3)

print(f"\n🏙️ Ciudades recuperadas:")
for idx, row in casos_ciudad_recuperada.iterrows():
    desc = str(row['description'])[:80] + "..."
    ciudad = row['l3_extracted']
    print(f"   → {ciudad}: {desc}")

print(f"\n✅ VALIDACIÓN COMPLETADA")
print(f"🏆 EXTRACCIÓN DE UBICACIÓN: ÉXITO TOTAL")
print(f"💎 Innovación de Text Mining: {mejora_ciudad + mejora_barrio:,} ubicaciones recuperadas")

🔍 VALIDACIÓN DE EXTRACCIÓN DE UBICACIÓN
📊 COMPARACIÓN ANTES vs DESPUÉS:
-----------------------------------
🏙️ CIUDADES:
   Antes: 274,167
   Después: 275,186
   Mejora: +1,019 (0.4%)

🏘️ BARRIOS:
   Antes: 65,980
   Después: 91,755
   Mejora: +25,775 (9.3%)

🗺️ IMPACTO EN COORDENADAS:
------------------------------
Coordenadas faltantes: 162,173
Barrios disponibles ahora: 91,755
🎯 Casos para imputar coords: 22,395

📋 MUESTRA DE CASOS EXITOSOS:
-----------------------------------
🏘️ Barrios recuperados:
   → El Poblado: Codigo Inmueble 5980 Hermosa casa para alquilar en zona residencial de Medellin,...
   → Belén: Codigo Inmueble 502031 CASA EN VENTA EN EL SECTOR DE BELÉN FATIMA CON 4 ALCOBAS,...
   → Laureles: Codigo Inmueble 561 Casa cerca al Éxito Laureles amplios espacios con muy buenas...
   → Laureles: Codigo Inmueble 608 Se vende casa en Laureles Almeria 120 mts. Sala comedor + ba...
   → Laureles: Codigo Inmueble 561 Casa cerca al Éxito Laureles amplios espacios con muy buenas.

In [42]:
# ===============================================================
# ANÁLISIS DEL CAMPO TITLE PARA EXTRACCIÓN DE BARRIOS
# ===============================================================

print("📰 ANÁLISIS DEL CAMPO TITLE PARA BARRIOS")
print("=" * 50)

# 1. Estado del campo title
print("📊 ESTADO DEL CAMPO TITLE:")
print("-" * 30)

title_disponible = df_clean['title'].notna().sum()
title_faltante = df_clean['title'].isna().sum()
total = len(df_clean)

print(f"Total registros: {total:,}")
print(f"Con title: {title_disponible:,} ({(title_disponible/total*100):.1f}%)")
print(f"Sin title: {title_faltante:,} ({(title_faltante/total*100):.1f}%)")

# 2. Candidatos para extracción desde title
candidatos_title = (
    df_clean['l4_final'].isna() & 
    df_clean['title'].notna()
).sum()

print(f"\n🎯 Candidatos title (sin barrio pero con title): {candidatos_title:,}")

# 3. Muestra de títulos para análisis
print(f"\n📝 MUESTRA DE TÍTULOS SIN BARRIO:")
print("-" * 35)

sin_barrio_con_title = df_clean[
    df_clean['l4_final'].isna() & 
    df_clean['title'].notna()
]['title'].sample(15, random_state=42)

for i, title in enumerate(sin_barrio_con_title, 1):
    title_corto = str(title)[:80] + "..." if len(str(title)) > 80 else str(title)
    print(f"\n{i:2d}. {title_corto}")

# 4. Búsqueda de barrios en títulos
print(f"\n\n🔍 BÚSQUEDA DE BARRIOS EN TÍTULOS:")
print("-" * 40)

# Lista de barrios conocidos (misma que usamos antes)
barrios_buscar_title = [
    'el poblado', 'poblado', 'laureles', 'belén', 'belen', 'la américa', 'america',
    'robledo', 'buenos aires', 'candelaria', 'san javier', 'guayabal', 'castilla',
    'san cristóbal', 'san cristobal', 'aranjuez', 'altavista', 'villa hermosa',
    'santa elena', 'san antonio de prado', 'manrique', 'doce de octubre', 
    'santa cruz', 'popular'
]

encontrados_title = {}
for barrio in barrios_buscar_title:
    # Patrón flexible para encontrar el barrio en título
    patron = rf'\b{re.escape(barrio)}\b'
    
    # Buscar en títulos SIN barrio
    matches = 0
    ejemplos = []
    
    for title in sin_barrio_con_title:
        if pd.notna(title):
            title_lower = str(title).lower()
            if re.search(patron, title_lower):
                matches += 1
                if len(ejemplos) < 2:  # Máximo 2 ejemplos
                    ejemplos.append(str(title)[:60] + "...")
    
    if matches > 0:
        encontrados_title[barrio] = {'matches': matches, 'ejemplos': ejemplos}

# Mostrar resultados
print(f"Barrios encontrados en muestra de {len(sin_barrio_con_title)} títulos:")
for barrio, info in encontrados_title.items():
    print(f"\n• {barrio}: {info['matches']} matches")
    for ejemplo in info['ejemplos']:
        print(f"  → {ejemplo}")

print(f"\n✅ Análisis de títulos completado")
print(f"📊 Barrios detectados en titles: {len(encontrados_title)} de {len(barrios_buscar_title)} buscados")

📰 ANÁLISIS DEL CAMPO TITLE PARA BARRIOS
📊 ESTADO DEL CAMPO TITLE:
------------------------------
Total registros: 277,856
Con title: 277,855 (100.0%)
Sin title: 1 (0.0%)

🎯 Candidatos title (sin barrio pero con title): 186,100

📝 MUESTRA DE TÍTULOS SIN BARRIO:
-----------------------------------

 1. VENDO APARTAMENTO AVES MARIAS SABANETA COD. 900871

 2. Apartamento en Venta Ubicado en MEDELLIN

 3. Apartamento en Arriendo Ubicado en SABANETA

 4. Apartamento en Arriendo Ubicado en MEDELLIN

 5. PR 12140 SE ARRIENDA APARTAMENTO EN SECTOR JARDINES _ ENVIGADO

 6. APARTAMENTO VENTA EL RETIRO ANTIOQUIA COD: 15101 _ wasi1260235

 7. Casa en Venta Ubicado en MEDELLIN

 8. Apartamento en Arriendo Ubicado en MEDELLIN

 9. Apartamento en Arriendo Ubicado en SABANETA

10. Loma del Escobero, venta apartamento

11. Casa en venta 90m2 Niquia Bello

12. Apartamento en Venta Ubicado en MEDELLIN

13. Apartamento en Arriendo Ubicado en RIONEGRO

14. Apartamento en arriendo en Rionegro (Antioquia) La 

In [43]:
# ===============================================================
# FUNCIÓN DE EXTRACCIÓN DESDE TITLES (CIUDADES Y SECTORES)
# ===============================================================

print("🔧 CREANDO FUNCIÓN DE EXTRACCIÓN DESDE TITLES")
print("=" * 50)

def extraer_ciudad_desde_title(title):
    """Extrae ciudad desde title usando patrones específicos"""
    if pd.isna(title):
        return None
    
    title_lower = str(title).lower()
    
    # Ciudades del área metropolitana
    ciudades_title = [
        'medellín', 'medellin', 'envigado', 'itagüí', 'itagui', 'sabaneta', 
        'bello', 'copacabana', 'la estrella', 'estrella', 'caldas', 
        'girardota', 'barbosa', 'rionegro'
    ]
    
    for ciudad in ciudades_title:
        patron = rf'\b{re.escape(ciudad)}\b'
        if re.search(patron, title_lower):
            # Mapear a nombre estándar
            if ciudad in ['medellin', 'medellín']:
                return 'Medellín'
            elif ciudad in ['itagui', 'itagüí']:
                return 'Itagüí'
            elif ciudad in ['estrella', 'la estrella']:
                return 'La Estrella'
            else:
                return ciudad.title()
    
    return None

def extraer_barrio_desde_title(title):
    """Extrae barrio/sector desde title con patrones específicos"""
    if pd.isna(title):
        return None
    
    title_lower = str(title).lower()
    
    # Barrios y sectores comunes en titles
    ubicaciones_title = [
        # Barrios conocidos
        'el poblado', 'poblado', 'laureles', 'belén', 'belen', 'la américa', 'america',
        'robledo', 'buenos aires', 'candelaria', 'san javier', 'guayabal', 'castilla',
        'san cristóbal', 'san cristobal', 'aranjuez', 'altavista', 'villa hermosa',
        'santa elena', 'san antonio de prado', 'manrique', 'doce de octubre', 
        'santa cruz', 'popular',
        # Sectores específicos que aparecen en titles
        'jardines', 'niquia', 'aves marias', 'loma del escobero', 'la rochela',
        'suramérica', 'suramerica', 'estadio', 'centro', 'boston', 'manila'
    ]
    
    for ubicacion in ubicaciones_title:
        patron = rf'\b{re.escape(ubicacion)}\b'
        if re.search(patron, title_lower):
            # Mapear a nombre estándar
            if ubicacion in ['poblado', 'el poblado']:
                return 'El Poblado'
            elif ubicacion in ['belen', 'belén']:
                return 'Belén'
            elif ubicacion in ['america', 'la américa']:
                return 'La América'
            elif ubicacion in ['san cristobal', 'san cristóbal']:
                return 'San Cristóbal'
            elif ubicacion in ['suramerica', 'suramérica']:
                return 'Suramérica'
            else:
                return ubicacion.title()
    
    return None

print("✅ Funciones de extracción desde title creadas")

# Probar funciones en muestra
print(f"\n🧪 PRUEBA EN MUESTRA DE TÍTULOS:")
print("-" * 35)

muestra_titles = df_clean['title'].dropna().sample(1000, random_state=42)

ciudades_title = muestra_titles.apply(extraer_ciudad_desde_title).notna().sum()
barrios_title = muestra_titles.apply(extraer_barrio_desde_title).notna().sum()

print(f"Ciudades desde title: {ciudades_title}/1000 ({(ciudades_title/1000*100):.1f}%)")
print(f"Barrios desde title: {barrios_title}/1000 ({(barrios_title/1000*100):.1f}%)")

# Ejemplos exitosos
print(f"\n📋 EJEMPLOS EXITOSOS DESDE TITLES:")
print("-" * 40)

ejemplos_ciudad_title = muestra_titles[muestra_titles.apply(extraer_ciudad_desde_title).notna()].head(3)
ejemplos_barrio_title = muestra_titles[muestra_titles.apply(extraer_barrio_desde_title).notna()].head(3)

print("🏙️ CIUDADES DESDE TITLE:")
for i, title in enumerate(ejemplos_ciudad_title, 1):
    ciudad = extraer_ciudad_desde_title(title)
    print(f"  {i}. {ciudad} ← {str(title)[:60]}...")

print("\n🏘️ BARRIOS/SECTORES DESDE TITLE:")
for i, title in enumerate(ejemplos_barrio_title, 1):
    barrio = extraer_barrio_desde_title(title)
    print(f"  {i}. {barrio} ← {str(title)[:60]}...")

print(f"\n✅ PRUEBAS COMPLETADAS")

🔧 CREANDO FUNCIÓN DE EXTRACCIÓN DESDE TITLES
✅ Funciones de extracción desde title creadas

🧪 PRUEBA EN MUESTRA DE TÍTULOS:
-----------------------------------
Ciudades desde title: 849/1000 (84.9%)
Barrios desde title: 129/1000 (12.9%)

📋 EJEMPLOS EXITOSOS DESDE TITLES:
----------------------------------------
🏙️ CIUDADES DESDE TITLE:
  1. Medellín ← Apartamento en Venta Ubicado en MEDELLIN...
  2. Medellín ← APARTAMENTO EN ARRIENDO, MEDELLIN-LOMA DE LOS BERNAL...
  3. Medellín ← Apartamento en Arriendo Ubicado en MEDELLIN...

🏘️ BARRIOS/SECTORES DESDE TITLE:
  1. El Poblado ← SE ARRIENDA APARTAESTUDIO EN SANTA MARIA DE LOS ANGELES , PO...
  2. El Poblado ← APARTAMENTO EN ARRIENDO, MEDELLIN-POBLADO...
  3. Estadio ← Apartamento en venta Estadio 106 mt² Exito Colombia...

✅ PRUEBAS COMPLETADAS
🏙️ CIUDADES DESDE TITLE:
  1. Medellín ← Apartamento en Venta Ubicado en MEDELLIN...
  2. Medellín ← APARTAMENTO EN ARRIENDO, MEDELLIN-LOMA DE LOS BERNAL...
  3. Medellín ← Apartamento en Arriend

In [45]:
# ===============================================================
# APLICACIÓN DE EXTRACCIÓN DESDE TITLES AL DATASET COMPLETO
# ===============================================================

print("🚀 EXTRACCIÓN DESDE TITLES - DATASET COMPLETO")
print("=" * 55)

# PASO 1: EXTRACCIÓN ADICIONAL DE CIUDADES DESDE TITLES
print("\n🏙️ EXTRACCIÓN ADICIONAL DE CIUDADES DESDE TITLES:")
print("-" * 55)

# Buscar registros que NO tienen ciudad pero SÍ tienen title
mask_sin_ciudad_con_title = (
    df_clean['l3_final'].isna() & 
    df_clean['title'].notna()
)

candidatos_ciudad_title = mask_sin_ciudad_con_title.sum()
print(f"Candidatos ciudad desde title: {candidatos_ciudad_title:,}")

if candidatos_ciudad_title > 0:
    # Extraer ciudades desde title
    extracciones_ciudad_title = df_clean.loc[mask_sin_ciudad_con_title, 'title'].apply(extraer_ciudad_desde_title)
    df_clean.loc[mask_sin_ciudad_con_title, 'l3_title_extracted'] = extracciones_ciudad_title
    
    ciudades_title_extraidas = df_clean['l3_title_extracted'].notna().sum()
    tasa_ciudad_title = (ciudades_title_extraidas / candidatos_ciudad_title * 100) if candidatos_ciudad_title > 0 else 0
    
    print(f"✅ Ciudades extraídas desde title: {ciudades_title_extraidas:,}")
    print(f"📈 Tasa de éxito: {tasa_ciudad_title:.1f}%")
    
    # Integrar ciudades desde title
    df_clean['l3_final'] = df_clean['l3_final'].fillna(df_clean['l3_title_extracted'])
else:
    print("No hay candidatos adicionales para ciudades desde title")
    ciudades_title_extraidas = 0

# PASO 2: EXTRACCIÓN ADICIONAL DE BARRIOS DESDE TITLES
print(f"\n🏘️ EXTRACCIÓN ADICIONAL DE BARRIOS DESDE TITLES:")
print("-" * 55)

# Buscar registros que NO tienen barrio pero SÍ tienen title
mask_sin_barrio_con_title = (
    df_clean['l4_final'].isna() & 
    df_clean['title'].notna()
)

candidatos_barrio_title = mask_sin_barrio_con_title.sum()
print(f"Candidatos barrio desde title: {candidatos_barrio_title:,}")

# Extraer barrios desde title
extracciones_barrio_title = df_clean.loc[mask_sin_barrio_con_title, 'title'].apply(extraer_barrio_desde_title)
df_clean.loc[mask_sin_barrio_con_title, 'l4_title_extracted'] = extracciones_barrio_title

barrios_title_extraidos = df_clean['l4_title_extracted'].notna().sum()
tasa_barrio_title = (barrios_title_extraidos / candidatos_barrio_title * 100) if candidatos_barrio_title > 0 else 0

print(f"✅ Barrios extraídos desde title: {barrios_title_extraidos:,}")
print(f"📈 Tasa de éxito: {tasa_barrio_title:.1f}%")

# Integrar barrios desde title
df_clean['l4_final'] = df_clean['l4_final'].fillna(df_clean['l4_title_extracted'])

# PASO 3: RESUMEN FINAL DE EXTRACCIÓN DESDE TITLES
print(f"\n📊 RESUMEN FINAL - EXTRACCIÓN DESDE TITLES:")
print("-" * 50)

print(f"🏙️ Ciudades adicionales desde title: {ciudades_title_extraidas:,}")
print(f"🏘️ Barrios adicionales desde title: {barrios_title_extraidos:,}")
print(f"🎯 Total ubicaciones desde title: {ciudades_title_extraidas + barrios_title_extraidos:,}")

# Estado final de ubicaciones
ciudades_finales = df_clean['l3_final'].notna().sum()
barrios_finales = df_clean['l4_final'].notna().sum()

print(f"\n📈 ESTADO FINAL DE UBICACIONES:")
print("-" * 35)
print(f"Ciudades finales: {ciudades_finales:,}")
print(f"Barrios finales: {barrios_finales:,}")

# Distribución de extracciones desde title
if 'l4_title_extracted' in df_clean.columns:
    print(f"\n📋 TOP BARRIOS EXTRAÍDOS DESDE TITLE:")
    print("-" * 40)
    top_barrios_title = df_clean['l4_title_extracted'].value_counts().head(10)
    for barrio, count in top_barrios_title.items():
        print(f"   {barrio:<20}: {count:>4,}")

if 'l3_title_extracted' in df_clean.columns:
    print(f"\n📋 TOP CIUDADES EXTRAÍDAS DESDE TITLE:")
    print("-" * 40)
    top_ciudades_title = df_clean['l3_title_extracted'].value_counts().head(10)
    for ciudad, count in top_ciudades_title.items():
        print(f"   {ciudad:<20}: {count:>4,}")

# Nuevo potencial de imputación de coordenadas
coords_faltantes_final = df_clean[['lat', 'lon']].isna().any(axis=1).sum()
barrios_disponibles_final = df_clean['l4_final'].notna().sum()

sin_coords_con_barrio_final = (
    df_clean[['lat', 'lon']].isna().any(axis=1) & 
    df_clean['l4_final'].notna()
).sum()

print(f"\n🗺️ IMPACTO FINAL EN COORDENADAS:")
print("-" * 35)
print(f"Coordenadas faltantes: {coords_faltantes_final:,}")
print(f"Barrios disponibles: {barrios_disponibles_final:,}")
print(f"🎯 Casos para imputar coords: {sin_coords_con_barrio_final:,}")

print(f"\n🏆 EXTRACCIÓN DESDE TITLES: COMPLETADA")
print(f"💎 Total innovación text mining: Description + Title")

🚀 EXTRACCIÓN DESDE TITLES - DATASET COMPLETO

🏙️ EXTRACCIÓN ADICIONAL DE CIUDADES DESDE TITLES:
-------------------------------------------------------
Candidatos ciudad desde title: 2,347
✅ Ciudades extraídas desde title: 322
📈 Tasa de éxito: 13.7%

🏘️ EXTRACCIÓN ADICIONAL DE BARRIOS DESDE TITLES:
-------------------------------------------------------
Candidatos barrio desde title: 179,623
✅ Barrios extraídos desde title: 6,477
📈 Tasa de éxito: 3.6%

📊 RESUMEN FINAL - EXTRACCIÓN DESDE TITLES:
--------------------------------------------------
🏙️ Ciudades adicionales desde title: 322
🏘️ Barrios adicionales desde title: 6,477
🎯 Total ubicaciones desde title: 6,799

📈 ESTADO FINAL DE UBICACIONES:
-----------------------------------
Ciudades finales: 275,508
Barrios finales: 98,232

📋 TOP BARRIOS EXTRAÍDOS DESDE TITLE:
----------------------------------------
   Suramérica          : 1,321
   El Poblado          : 1,314
   Niquia              : 1,206
   Loma Del Escobero   : 1,204
   Cen